# Simulations
 - For mass testing strategy on random stocks

### Imports

In [11]:
import numpy as np

from data_collection import get_sp500_sample, get_historical_data
from computing import compute_momentum, compute_mean_reversion, compute_signal_returns
from signals import generate_signals_momentum, generate_signals_meanreversion, generate_final_signal
from trading import build_strategy_choice_label, build_feature_matrix, track_strategy_chosen_signals
from training import train_strategy_chooser
from risk_management import apply_stop_loss

### Parameters

In [12]:
# PARAMETERS
n_simulations = 20      # Run different random samples
sample_size = 15        # Number of stocks to sample (excluding SPY)
start_date = "2015-01-01"
end_date = "2025-01-01"
initial_capital = 10000.0

### Running Sims

In [13]:
# Containers for final portfolio values (from the last date in each simulation)
final_strategy_values = []
final_spy_values = []
final_equal_values = []

for sim in range(n_simulations):
    print(f"\n--- Running Simulation {sim+1} of {n_simulations} ---")
    # Get a random sample of S&P 500 tickers
    tickers = get_sp500_sample(n=sample_size)
    # Always add SPY as benchmark
    if "SPY" not in tickers:
        tickers.append("SPY")
    
    # Download historical data for the chosen tickers
    data = get_historical_data(tickers, start=start_date, end=end_date)
    if data.empty or data.shape[0] < 100:
        print("Insufficient data returned for this sample. Skipping simulation.")
        continue

    # Compute momentum and mean-reversion (z-score) using your windows
    momentum_df = compute_momentum(data, window=63)    # About 3 months of trading days
    zscore_df   = compute_mean_reversion(data, window=20)  # 20-day rolling window

    # Generate signals for each approach
    momentum_signals = generate_signals_momentum(momentum_df, threshold=0.10)
    meanrev_signals  = generate_signals_meanreversion(zscore_df, z_entry=1.0)

    # Compute next-day returns based on the signals (used for choosing the best strategy)
    momentum_returns = compute_signal_returns(data, momentum_signals)
    meanrev_returns  = compute_signal_returns(data, meanrev_signals)

    # 1 if momentum outperformed mean-reversion on that day, 0 otherwise.
    label = build_strategy_choice_label(momentum_returns, meanrev_returns)

    # Build features for the strategy chooser (averaging factors across stocks, plus volatility etc.)
    features = build_feature_matrix(data, momentum_df, zscore_df, vol_window=20)

    # TRAIN THE STRATEGY CHOOSER
    try:
        clf = train_strategy_chooser(features, label)
    except Exception as e:
        print(f"Training classifier failed with error: {e}\nSkipping simulation.")
        continue

    #GENERATE FINAL SIGNALS & APPLY STOP-LOSS
    signals = generate_final_signal(clf, features, momentum_signals, meanrev_signals)
    final_signals = apply_stop_loss(signals, data, stop_loss_pct=0.05)

    # TRACK PERFORMANCE
    # Use the SPY column from your data as the benchmark series
    spy_series = data["SPY"]
    result_df = track_strategy_chosen_signals(data, final_signals, spy_series, initial_capital=initial_capital)
    
    # Make sure we have results; use the last available date to record final portfolio values.
    if result_df.empty:
        print("No results available from tracking performance. Skipping simulation.")
        continue

    final_date = result_df.index[-1]
    final_strategy = result_df.at[final_date, "Strategy"]
    final_spy = result_df.at[final_date, "SPY_BuyHold"]
    final_equal = result_df.at[final_date, "EqualWeight_BuyHold"]

    final_strategy_values.append(final_strategy)
    final_spy_values.append(final_spy)
    final_equal_values.append(final_equal)

    print(f"Simulation {sim+1} final values (Date: {final_date.date()}):")
    print(f"  ML Strategy:           ${final_strategy:,.2f}")
    print(f"  SPY Buy & Hold:        ${final_spy:,.2f}")
    print(f"  Equal Weight Buy & Hold: ${final_equal:,.2f}")

# COMPUTE AVERAGES AND MEDIANS ACROSS SIMULATIONS
if final_strategy_values:
    avg_strategy = np.mean(final_strategy_values)
    avg_spy = np.mean(final_spy_values)
    avg_equal = np.mean(final_equal_values)

    median_strategy = np.median(final_strategy_values)
    median_spy = np.median(final_spy_values)
    median_equal = np.median(final_equal_values)

    max_strategy = np.max(final_strategy_values)
    min_strategy = np.min(final_strategy_values)

    print("\n============================================")
    print("Final Portfolio Statistics (over simulations):")
    print(f"  ML Strategy - Average:            ${avg_strategy:,.2f}")
    print(f"  SPY Buy & Hold - Average:         ${avg_spy:,.2f}")
    print(f"  Equal Weight Buy & Hold - Average: ${avg_equal:,.2f}")
    print()
    print(f"  ML Strategy - Median:             ${median_strategy:,.2f}")
    print(f"  SPY Buy & Hold - Median:          ${median_spy:,.2f}")
    print(f"  Equal Weight Buy & Hold - Median:  ${median_equal:,.2f}")
    print("============================================\n")
    print()
    print(f"  ML Strategy - Max:                ${max_strategy:,.2f}")
    print(f"  ML Strategy - Min:                ${min_strategy:,.2f}")
    print("============================================\n")
else:
    print("No successful simulations to report statistics.")


--- Running Simulation 1 of 20 ---


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Best Params: {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 50}
Best CV Score: 0.5198979591836734
Simulation 1 final values (Date: 2024-12-31):
  ML Strategy:           $140,509.81
  SPY Buy & Hold:        $33,957.37
  Equal Weight Buy & Hold: $43,385.35

--- Running Simulation 2 of 20 ---


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Best Params: {'max_depth': 3, 'min_samples_split': 5, 'n_estimators': 100}
Best CV Score: 0.49744897959183676
Simulation 2 final values (Date: 2024-12-31):
  ML Strategy:           $28,919.99
  SPY Buy & Hold:        $33,957.37
  Equal Weight Buy & Hold: $38,534.25

--- Running Simulation 3 of 20 ---


...

Best Params: {'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 100}
Best CV Score: 0.5183673469387755


[*********************100%***********************]  1 of 1 completed

Simulation 19 final values (Date: 2024-12-31):
  ML Strategy:           $27,166.17
  SPY Buy & Hold:        $33,957.37
  Equal Weight Buy & Hold: $35,095.52

--- Running Simulation 20 of 20 ---



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******

Best Params: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 100}
Best CV Score: 0.5137755102040816
Simulation 20 final values (Date: 2024-12-31):
  ML Strategy:           $10,669,374.00
  SPY Buy & Hold:        $33,957.37
  Equal Weight Buy & Hold: $32,123.73

Final Portfolio Statistics (over simulations):
  ML Strategy - Average:            $8,947,710.97
  SPY Buy & Hold - Average:         $33,957.37
  Equal Weight Buy & Hold - Average: $40,917.54

  ML Strategy - Median:             $3,909,926.02
  SPY Buy & Hold - Median:          $33,957.37
  Equal Weight Buy & Hold - Median:  $41,654.25


  ML Strategy - Max:                $63,902,415.36
  ML Strategy - Min:                $25,567.26

